In [1132]:
import re
from datetime import datetime, timedelta
import pandas as pd

In [1133]:
# 날짜 범위 생성
# 현재 날짜 가져오기
today = datetime.today()

if today <  datetime.strptime('2024-03-15', '%Y-%m-%d'):
    today = '2024-03-15'

date_rng = pd.date_range(start='2024-02-19', end=today, freq='D')

# 데이터프레임 생성
설정맵 = pd.DataFrame(columns=['박연수', '권세한', '권하은',
                            '박지은', '박하은', '손하진',
                            '송호석', '이신후', '이준수', '임혁규'], index=date_rng)


벌금맵 = 설정맵

설정맵 = 설정맵.fillna("설정X")
벌금맵 = 벌금맵.fillna(0)

설정맵.loc['2024-02-19'] = '성공'

for idx, row in 설정맵.iterrows():
    if idx.weekday() == 6:  # 일요일인 경우
        설정맵.loc[idx] = 'off'

In [1134]:
file = pd.read_csv("6기.csv", encoding='UTF8')

pattern_set = r'\d{4}\s?\w+\s?설정' # 설정 정규식
pattern_success = r'\d{4}\s?\w+\s?성공' # 성공 정규식
pattern_fail = r'\d{4}\s?\w+\s?실패' # 실패 정규식
pattern_off = r'\d{4}\s?\w+\s?.*?\s?off' # off 정규식
pattern_offs = r'\d{4}-\d{4}\s?\w+\s?.*?\s?off' # offs 정규식
pattern_retry_set = r'\d{4}\s?\w+\s?\d{4}\s?재도전' # 재도전 설정 정규식
pattern_retry_sucess = r'\d{4}\s?\w+\s?\d{4}\s?재성공' # 재도전 인증 정규식

filter_option = f"{pattern_set}|{pattern_success}|{pattern_fail}|{pattern_off}|{pattern_offs}|{pattern_retry_set}|{pattern_retry_sucess}"
filtered_rows = file[file['Message'].str.contains(filter_option, regex=True)]
filtered_rows = filtered_rows.reset_index(drop=True, inplace=False)
filtered_rows['Late'] = 'O'

len_of_rows = filtered_rows.shape[0]
# filtered_rows

In [1135]:
def deadline_set(USER, line):
    if USER in ['권세한', '임혁규', '이신후', '이준수']:
        DEADLINE = 21
    elif 3 <= datetime.strptime(line, "%Y%m%d").month <= 6:
        DEADLINE = 11    
    else :
        DEADLINE = 10
        
    return DEADLINE
        

In [1136]:
# 지각 여부 확인
i = 0
for index in range(len_of_rows):
    DATE = filtered_rows['Date'][index]
    USER = filtered_rows['User'][index]
    MESSAGE = filtered_rows['Message'][index]
    LATE = filtered_rows['Late'][index]



    result_set = str(re.findall(pattern_set, MESSAGE))[2:][:-2]
    result_retry_set = str(re.findall(pattern_retry_set, MESSAGE))[2:][:-2]

    result_success = str(re.findall(pattern_success, MESSAGE))[2:][:-2]
    result_retry_success = str(re.findall(pattern_retry_sucess, MESSAGE))[2:][:-2]
    result_fail = str(re.findall(pattern_fail, MESSAGE))[2:][:-2]
        
        
    time_str = DATE
    
    time_obj = datetime.strptime(time_str, '%Y-%m-%d %H:%M:%S')

    
    if len(result_success) >= 1 | len(result_fail) >= 1 | len(result_retry_success) >= 1:
        if  len(result_retry_success) > 1:
            date_str_next = '2024' + result_retry_success[:4]
            
        elif len(result_fail) > 1:
            date_str_next = '2024' + result_fail[:4]
        
        else:
            date_str_next = '2024' + result_success[:4]

        DEADLINE = deadline_set(USER, date_str_next)
            
        date_obj_next = datetime.strptime(date_str_next + f'{DEADLINE}', "%Y%m%d%H") + timedelta(days=1)
        formatted_date_next = date_obj_next.strftime("%Y-%m-%d %H:%M:%S")
        time_difference_next = time_obj - datetime.strptime(formatted_date_next, "%Y-%m-%d %H:%M:%S")    
        date_str_next = date_obj_next.strftime("%Y-%m-%d")
        date_index_next = datetime.strptime(date_str_next, '%Y-%m-%d')
    
        if time_difference_next.days > 0 or (time_difference_next.days == 0 and time_difference_next.seconds >= 60):
            filtered_rows['Late'][index] = '지각'
        
    
    elif len(result_set) >= 1 | len(result_retry_set) >= 1:
        if len(result_retry_set) > 1:
            date_str = '2024' + result_retry_set[:4]
        else:
            date_str = '2024' + result_set[:4]
            
        DEADLINE = deadline_set(USER, date_str)

        date_obj = datetime.strptime(date_str + f'{DEADLINE}', "%Y%m%d%H")
        formatted_date = date_obj.strftime("%Y-%m-%d %H:%M:%S")
        time_difference = time_obj - datetime.strptime(formatted_date, "%Y-%m-%d %H:%M:%S")
        date_str = date_obj.strftime("%Y-%m-%d")
        date_index = datetime.strptime(date_str, '%Y-%m-%d')
    
        if time_difference.days > 0 or (time_difference.days == 0 and time_difference.seconds >= 60):
            filtered_rows['Late'][index] = '지각'

In [1137]:
filtered_rows

,Date,User,Message,Late
0,2024-02-19 00:40:35,박지은,0219-0220 박지은 여행으로 off,O
1,2024-02-19 03:19:04,권하은,0129-0220 권하은 새터 및 학생회 사업으로 off,O
2,2024-02-19 03:23:12,권세한,0219 권세한 복귀이슈로 off,O
3,2024-02-19 05:36:24,이준수,0219-0224 휴가로 off,O
4,2024-02-19 13:15:49,이준수,0219-0224 이준수 휴가로 off,O
...,...,...,...,...
157,2024-03-05 22:52:21,박지은,0306 박지은 감기몸살로 off,O
158,2024-03-06 01:02:24,박하은,0305 박하은 성공\n0306 박하은 설정\n- 취업 사이트에서 현황 분석\n- ...,O
159,2024-03-06 10:41:00,송호석,0305 송호석 실패\n0306 송호석 설정\n-클립스튜디오 브러쉬 파악\n-그림체 연구,O
160,2024-03-06 10:41:01,박연수,0305 박연수 성공\n0306 박연수 설정\n- [ ] 알고리즘 강의 1개 \n-...,O


In [1138]:
#설정 코드
for index in range(len_of_rows):
    DATE = filtered_rows['Date'][index]
    USER = filtered_rows['User'][index]
    MESSAGE = filtered_rows['Message'][index]
    LATE = filtered_rows['Late'][index]

    result_set = str(re.findall(pattern_set, MESSAGE))[2:][:-2]
    result_retry_set = str(re.findall(pattern_retry_set, MESSAGE))[2:][:-2]

    if len(result_set) <= 1 | len(result_retry_set) <= 1:
        continue
    elif len(result_retry_set) > 1:
        date_str = '2024' + result_retry_set[:4]
    else:
        date_str = '2024' + result_set[:4]

    DEADLINE = deadline_set(USER, date_str)

    date_obj = datetime.strptime(date_str + f'{DEADLINE}', "%Y%m%d%H")

    time_str = DATE
    time_obj = datetime.strptime(time_str, '%Y-%m-%d %H:%M:%S')

    formatted_date = date_obj.strftime("%Y-%m-%d %H:%M:%S")

    time_difference = time_obj - datetime.strptime(formatted_date, "%Y-%m-%d %H:%M:%S")

    # 결과 확인
    date_str = date_obj.strftime("%Y-%m-%d")

    date_index = datetime.strptime(date_str, '%Y-%m-%d')
    
    if filtered_rows['Late'][index] == '지각':
        설정맵.loc[date_index, USER] = '지각'
    else: 
        설정맵.loc[date_index, USER] = 'O'

ValueError: time data '2024-03-06' does not match format '%Y%m%d'

In [ ]:
설정맵

,박연수,권세한,권하은,박지은,박하은,손하진,송호석,이신후,이준수,임혁규
2024-02-19,성공,성공,성공,성공,성공,성공,성공,성공,성공,성공
2024-02-20,O,O,설정X,설정X,O,O,O,O,설정X,O
2024-02-21,O,O,지각,O,O,O,O,O,설정X,지각
2024-02-22,O,O,설정X,O,O,지각,O,O,설정X,설정X
2024-02-23,O,설정X,O,설정X,O,O,설정X,O,설정X,설정X
2024-02-24,O,O,O,O,O,O,설정X,설정X,설정X,설정X
2024-02-25,off,off,지각,off,off,off,off,off,O,off
2024-02-26,설정X,O,O,O,O,지각,O,설정X,O,설정X
2024-02-27,설정X,설정X,O,O,O,설정X,O,설정X,O,설정X
2024-02-28,O,설정X,O,O,O,O,지각,설정X,O,설정X


In [ ]:
# 성공 코드
for index in range(len_of_rows):
    DATE = filtered_rows['Date'][index]
    USER = filtered_rows['User'][index]
    MESSAGE = filtered_rows['Message'][index]
    LATE = filtered_rows['Late'][index]

    result_success = str(re.findall(pattern_success, MESSAGE))[2:][:-2]
    result_retry_success = str(re.findall(pattern_retry_sucess, MESSAGE))[2:][:-2]
    result_fail = str(re.findall(pattern_fail, MESSAGE))[2:][:-2]

    if len(result_success) <= 1 | len(result_fail) <= 1 | len(result_retry_success) <= 1:
        continue
    
    if  len(result_retry_success) > 1:
        re.search(pattern_retry_sucess, result_retry_success)
        date_range_str = re.search(r'\d{4}\s?\w+\s?\d{4}', result_retry_success).group()
        date_str = datetime.strptime('2024'+date_range_str[:4], "%Y%m%d")
        target_date = datetime.strptime('2024'+date_range_str[-4:], "%Y%m%d")
        

        if 설정맵.loc[date_str, USER] == 'O':
            설정맵.loc[date_str, USER] = '성공'
            설정맵.loc[target_date, USER] = '재성공'
            continue

        elif 설정맵.loc[date_str, USER] == '지각':
            설정맵.loc[date_str, USER] = '성공'
            설정맵.loc[target_date, USER] = '지각+재성공'
            continue
        
    elif len(result_fail) >= 1:
        date_str = '2024' + result_fail[:4]
    
    else:
        date_str = '2024' + result_success[:4]
        
    date_obj = datetime.strptime(date_str, "%Y%m%d")

    date_str = date_obj.strftime("%Y-%m-%d")

    if filtered_rows['Late'][index]=='지각':
        if len(result_success)  > 1:
            설정맵.loc[date_str, USER] = '지각+성공'
            continue
        
        elif len(result_fail)  > 1: 
            설정맵.loc[date_str, USER] = '지각+실패'
            continue
    
    elif 설정맵.loc[date_str, USER] == 'O':
        if len(result_success)  > 1:
            설정맵.loc[date_str, USER] = '성공'
            continue
        
        elif len(result_fail)  > 1: 
            설정맵.loc[date_str, USER] = '실패'
            continue
        
    elif 설정맵.loc[date_str, USER] == '지각':
        if len(result_success)  > 1:
            설정맵.loc[date_str, USER] = '지각+성공'
            continue
        
        elif len(result_fail)  > 1: 
            설정맵.loc[date_str, USER] = '지각+실패'
            continue

In [ ]:
설정맵

,박연수,권세한,권하은,박지은,박하은,손하진,송호석,이신후,이준수,임혁규
2024-02-19,성공,성공,성공,성공,성공,성공,성공,성공,성공,성공
2024-02-20,성공,실패,설정X,설정X,성공,실패,성공,O,설정X,지각+성공
2024-02-21,성공,성공,지각+실패,실패,성공,성공,성공,지각+성공,설정X,지각+성공
2024-02-22,성공,성공,설정X,실패,O,지각+성공,실패,성공,설정X,설정X
2024-02-23,성공,설정X,성공,설정X,실패,실패,설정X,실패,설정X,설정X
2024-02-24,성공,실패,지각+실패,실패,실패,O,설정X,설정X,설정X,설정X
2024-02-25,off,off,지각+성공,off,off,off,off,off,성공,off
2024-02-26,설정X,성공,성공,성공,성공,지각+실패,성공,설정X,성공,설정X
2024-02-27,설정X,설정X,성공,성공,실패,설정X,성공,설정X,성공,설정X
2024-02-28,실패,설정X,성공,지각+성공,성공,지각+성공,지각+실패,설정X,성공,설정X


In [ ]:
# off 코드
for index in range(len_of_rows):
    DATE = filtered_rows['Date'][index]
    USER = filtered_rows['User'][index]
    MESSAGE = filtered_rows['Message'][index]

    result_off = str(re.findall(pattern_off, MESSAGE))[2:][:-2]
    result_offs = str(re.findall(pattern_offs, MESSAGE))[2:][:-2]

    if len(result_off) <= 1:
        continue
    else:
        date_str = '2024' + result_off[:4]
        
    date_obj = datetime.strptime(date_str, "%Y%m%d")

    date_str = date_obj.strftime("%Y-%m-%d")
    
    if (result_off[-3:] == 'off') & ((설정맵.loc[date_str, USER] == 'O') | (설정맵.loc[date_str, USER] == '설정X')):
        설정맵.loc[date_str, USER] = 'off'
        print(result_off)
        
    if re.search(pattern_offs, result_offs):
        date_range_str = re.search(r'\d{4}-\d{4}', result_offs).group()
        start_date = datetime.strptime('2024'+date_range_str[:4], "%Y%m%d")
        end_date = datetime.strptime('2024'+date_range_str[5:9], "%Y%m%d")
        
        설정맵.loc[start_date:end_date, USER] = 'off'

0220 박지은 여행으로 off
0220 권하은 새터 및 학생회 사업으로 off
0224 휴가로 off
0222 권하은 몸살로 off
0222 임혁규 off
0223송호석 off
0223 권세한 외박출타로 off
0223 박지은 졸업식으로 off
0222 박하은 할머니 병원으로 off
0220 이신후 off
0223 임혁규 off
0224 이신후 off
0224 임혁규 off
0224 송호석 off
0229 이신후 off
0228 임혁규 off
0226 박연수 off
0227 박연수 off
0227 손하진 off
0315 권세한 UFS 훈련으로 off
0229 임혁규 off
0301 송호석 off
0301 박하은 off
0301 권하은 몸살로 off
0302 권하은 몸살로 off
0302 임혁규 off
0302 이신후 off
0302 송호석 off
0308 이준수 훈련으로 off
0307 임혁규 off
0304 이신후 off
0305 박지은 off
0306 박지은 감기몸살로 off


In [ ]:
설정맵

,박연수,권세한,권하은,박지은,박하은,손하진,송호석,이신후,이준수,임혁규
2024-02-19,성공,성공,off,off,성공,성공,성공,성공,off,성공
2024-02-20,성공,실패,off,off,성공,실패,성공,off,off,지각+성공
2024-02-21,성공,성공,지각+실패,실패,성공,성공,성공,지각+성공,off,지각+성공
2024-02-22,성공,성공,off,실패,off,지각+성공,실패,성공,off,off
2024-02-23,성공,off,성공,off,실패,실패,off,실패,off,off
2024-02-24,성공,실패,지각+실패,실패,실패,O,off,off,off,off
2024-02-25,off,off,지각+성공,off,off,off,off,off,성공,off
2024-02-26,off,성공,성공,성공,성공,지각+실패,성공,off,성공,off
2024-02-27,off,off,성공,성공,실패,off,성공,off,성공,off
2024-02-28,실패,off,성공,지각+성공,성공,지각+성공,지각+실패,off,성공,off


In [ ]:
# 5일 연속 성공 여부 확인
def check_consecutive_success(user_data):
    consecutive_count = 0
    for status in user_data:
        if status == '성공':
            consecutive_count += 1
            if consecutive_count == 5:
                return True
        else:
            consecutive_count = 0
    return False

# 부여 까방권
for user in 설정맵.columns:
    user_data = 설정맵[user].dropna()  # NaN 값 제외
    if check_consecutive_success(user_data):
        print(f'{user}님에게 까방권 부여!')

박연수님에게 까방권 부여!
이준수님에게 까방권 부여!


In [ ]:
설정맵

,박연수,권세한,권하은,박지은,박하은,손하진,송호석,이신후,이준수,임혁규
2024-02-19,성공,성공,off,off,성공,성공,성공,성공,off,성공
2024-02-20,성공,실패,off,off,성공,실패,성공,off,off,지각+성공
2024-02-21,성공,성공,지각+실패,실패,성공,성공,성공,지각+성공,off,지각+성공
2024-02-22,성공,성공,off,실패,off,지각+성공,실패,성공,off,off
2024-02-23,성공,off,성공,off,실패,실패,off,실패,off,off
2024-02-24,성공,실패,지각+실패,실패,실패,O,off,off,off,off
2024-02-25,off,off,지각+성공,off,off,off,off,off,성공,off
2024-02-26,off,성공,성공,성공,성공,지각+실패,성공,off,성공,off
2024-02-27,off,off,성공,성공,실패,off,성공,off,성공,off
2024-02-28,실패,off,성공,지각+성공,성공,지각+성공,지각+실패,off,성공,off


In [ ]:
max_consecutive_failures = 3
max_consecutive_successes = 3
max_consecutive_late_failures = 3
max_consecutive_late_sucesses = 3
# 설정맵을 순회하면서 벌금맵에 적절한 값 설정
for person in 설정맵.columns:
    FAIL_STATE = 3000
    LATE_STATE = 1000

    # 연속으로 나온 실패 및 성공 횟수 초기화
    consecutive_failures = 0
    consecutive_successes = 0
    consecutive_late_successes = 0
    consecutive_late_failures = 0

    for date, status in 설정맵[person].items():
        if pd.notna(status):  # NaN이 아닌 경우에만 처리
            time_str = str(date)
            time_obj = datetime.strptime(time_str, '%Y-%m-%d %H:%M:%S')

            formatted_date = date_obj.strftime("%Y-%m-%d %H:%M:%S")

            time_difference = time_obj - datetime.today()
            
            if '지각' in str(status):  # '지각'이 문자열에 포함되어 있는지 확인
                벌금맵[person][date] += LATE_STATE
                consecutive_late_failures += 1
                consecutive_late_successes = 0

            else : 
                consecutive_late_successes += 1
                consecutive_late_failures = 0
                
            if '성공' in str(status):
                벌금맵[person][date] += 0
                consecutive_failures = 0
                consecutive_successes += 1
                
                
            elif '실패' in str(status):
                벌금맵[person][date] += FAIL_STATE
                consecutive_failures += 1
                consecutive_successes = 0
    
                
            elif status == 'off':
                벌금맵[person][date] = 0

            elif status == '재성공':
                벌금맵[person][date] = 0
                consecutive_failures = 0
                consecutive_successes += 1

            elif status == '설정X' and  time_difference.days < -2:
                벌금맵[person][date] = FAIL_STATE+LATE_STATE
                consecutive_late_failures += 1
                consecutive_failures += 1
                consecutive_successes = 0
    
            elif status == 'O' and  time_difference.days < -2 :    
                벌금맵[person][date] = FAIL_STATE+LATE_STATE
                consecutive_late_failures += 1
                consecutive_failures += 1
                consecutive_successes = 0
    
            # 연속으로 나온 실패 및 성공 횟수가 설정값 이상이면 벌금 증가 또는 감소
            if consecutive_failures >= max_consecutive_failures:
                FAIL_STATE = 5000
                
            elif consecutive_successes >= max_consecutive_successes:
                FAIL_STATE = 3000

            if consecutive_late_failures >= max_consecutive_late_failures:
                LATE_STATE = 2000
                
            elif consecutive_late_successes >= max_consecutive_late_sucesses:
                LATE_STATE = 1000
                    
                


In [ ]:
벌금맵

,박연수,권세한,권하은,박지은,박하은,손하진,송호석,이신후,이준수,임혁규
2024-02-19,0,0,0,0,0,0,0,0,0,0
2024-02-20,0,3000,0,0,0,3000,0,0,0,1000
2024-02-21,0,0,4000,3000,0,0,0,1000,0,1000
2024-02-22,0,0,0,3000,0,1000,3000,0,0,0
2024-02-23,0,0,0,0,3000,3000,0,3000,0,0
2024-02-24,0,3000,4000,3000,3000,4000,0,0,0,0
2024-02-25,0,0,1000,0,0,0,0,0,0,0
2024-02-26,0,0,0,0,0,4000,0,0,0,0
2024-02-27,0,0,0,0,3000,0,0,0,0,0
2024-02-28,3000,0,0,1000,0,1000,4000,0,0,0


In [ ]:
# datetime 인덱스를 'YYYY-MM-DD' 포맷으로 변경
벌금맵.index = 벌금맵.index.map(lambda x: x.strftime('%Y-%m-%d') if isinstance(x, pd.Timestamp) else x)
설정맵.index = 설정맵.index.map(lambda x: x.strftime('%Y-%m-%d') if isinstance(x, pd.Timestamp) else x)

벌금맵.loc['합계'] = 벌금맵.sum()
# 각 열의 합을 계산하여 마지막 행에 추가
                        # 박연수 권세한  권하은  박지은  박하은  손하진  송호석  이신후  이준수  임혁규
벌금맵.loc['입금']        = 3000, 6000, 9000, 9000, 9000,  0,   0,  3000,  0,  2000
벌금맵.loc['벌금 연체료']   = 0000, 0000, 0000, 0000, 0000,  10_000,   10_000,  10_000,  0000,  0000
벌금맵.loc['미납'] = 벌금맵.loc['합계'] + 벌금맵.loc['벌금 연체료'] - 벌금맵.loc['입금']
벌금맵.loc['지원금'] = 0
벌금맵.loc[''] = ' '

# 총 벌금
total_penalty = 벌금맵.loc['합계'].sum() + 275_992
deposit_penalty = 벌금맵.loc['입금'].sum() + 275_992

# off 제외 설정맵의 개수
total_cells_excluding_off = 설정맵.apply(lambda x: x != 'off').sum().sum()
print("off를 제외한 셀의 개수:", total_cells_excluding_off)

for col in 설정맵.columns:
    col_off_count = 설정맵[col].apply(lambda x: x != 'off').sum()
    subsidy_proportion = col_off_count / total_cells_excluding_off
    subsidy =int(round(subsidy_proportion * total_penalty, -2))
    print(f"{col}의 예상 지원금: {subsidy}원")
    벌금맵[col]['지원금'] = subsidy
    print("마진",벌금맵[col]['지원금']-벌금맵[col]['합계'])
    

total_subsidy = 벌금맵.loc['지원금'].sum()
    
print("총 벌금 :" , total_penalty)
print("총 지원금 :" , total_subsidy)
print("총 입금 :" , deposit_penalty)

# 출력
벌금맵

off를 제외한 셀의 개수: 168
박연수의 예상 지원금: 45700원
마진 42700
권세한의 예상 지원금: 13100원
마진 7100
권하은의 예상 지원금: 41400원
마진 31400
박지은의 예상 지원금: 41400원
마진 19400
박하은의 예상 지원금: 47900원
마진 35900
손하진의 예상 지원금: 47900원
마진 29900
송호석의 예상 지원금: 41400원
마진 31400
이신후의 예상 지원금: 32700원
마진 28700
이준수의 예상 지원금: 28300원
마진 25300
임혁규의 예상 지원금: 26100원
마진 24100
총 벌금 : 365992
총 지원금 : 365900
총 입금 : 316992


,박연수,권세한,권하은,박지은,박하은,손하진,송호석,이신후,이준수,임혁규
2024-02-19,0,0,0,0,0,0,0,0,0,0
2024-02-20,0,3000,0,0,0,3000,0,0,0,1000
2024-02-21,0,0,4000,3000,0,0,0,1000,0,1000
2024-02-22,0,0,0,3000,0,1000,3000,0,0,0
2024-02-23,0,0,0,0,3000,3000,0,3000,0,0
2024-02-24,0,3000,4000,3000,3000,4000,0,0,0,0
2024-02-25,0,0,1000,0,0,0,0,0,0,0
2024-02-26,0,0,0,0,0,4000,0,0,0,0
2024-02-27,0,0,0,0,3000,0,0,0,0,0
2024-02-28,3000,0,0,1000,0,1000,4000,0,0,0


In [ ]:
설정맵

,박연수,권세한,권하은,박지은,박하은,손하진,송호석,이신후,이준수,임혁규
2024-02-19,성공,성공,off,off,성공,성공,성공,성공,off,성공
2024-02-20,성공,실패,off,off,성공,실패,성공,off,off,지각+성공
2024-02-21,성공,성공,지각+실패,실패,성공,성공,성공,지각+성공,off,지각+성공
2024-02-22,성공,성공,off,실패,off,지각+성공,실패,성공,off,off
2024-02-23,성공,off,성공,off,실패,실패,off,실패,off,off
2024-02-24,성공,실패,지각+실패,실패,실패,O,off,off,off,off
2024-02-25,off,off,지각+성공,off,off,off,off,off,성공,off
2024-02-26,off,성공,성공,성공,성공,지각+실패,성공,off,성공,off
2024-02-27,off,off,성공,성공,실패,off,성공,off,성공,off
2024-02-28,실패,off,성공,지각+성공,성공,지각+성공,지각+실패,off,성공,off


In [ ]:
맵 = pd.concat((벌금맵, 설정맵))

맵.to_csv('결과.csv')